In [ ]:
#default_exp data.datasets.m5

# M5 dataset

> Download and evaluate the M5 dataset.

The 2020 M5 competition was organized by the Makridakis Open Forecasting Center was organized by University of Nicosia and hosted in Kaggle. The main task consisted in providing hierarchical point and probabilistic predictions for 28 days ahead for the sales of Walmart stores in three US States (California, Texas, and Wisconsin). The dataset is organized at the item, department, product and store level details. Additionally it contains temporal covariates like price, promotions, special event and calendar variables.


[University of Nicosia. The M5 competition: Estimate the unit sales of Walmart retail goods. Kaggle Competition, 2020.](https://www.kaggle.com/c/m5-forecasting-accuracy/)

In [ ]:
#export
import os
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd

from nixtlats.data.datasets.utils import download_file, Info

In [ ]:
#export
@dataclass
class M5:
    
    # original data available from Kaggle directly
    # pip install kaggle --upgrade
    # kaggle competitions download -c m5-forecasting-accuracy
    source_url = 'https://www.dropbox.com/s/u5e0fo5yn76eri8/m5-forecasting-accuracy.zip?dl=1'
    
    @staticmethod
    def download(directory: str) -> None:
        """Downloads M5 Competition Dataset."""
        path = f'{directory}/m5/datasets'
        if not os.path.exists(path):
            download_file(directory=path,
                          source_url=M5.source_url,
                          decompress=True)
            
    @staticmethod
    def load(directory: str, cache: bool = True) -> Tuple[pd.DataFrame, 
                                                          pd.DataFrame, 
                                                          pd.DataFrame]:
        """Downloads and loads M5 data.

        Parameters
        ----------
        directory: str
            Directory where data will be downloaded.
        cache: bool
            If `True` saves and loads 
            
        Notes
        -----
        [1] Returns train+test sets.
        [2] Based on https://www.kaggle.com/lemuz90/m5-preprocess.
        """
        path = f'{directory}/m5/datasets'
        file_cache = f'{path}/m5.p'
        
        if os.path.exists(file_cache) and cache:
            Y_df, X_df, S_df = pd.read_pickle(file_cache)
            
            return Y_df, X_df, S_df
        
        M5.download(directory)
        # Calendar data
        cal_dtypes = {
            'd': 'category',
            'wm_yr_wk': np.uint16,
            'event_name_1': 'category',
            'event_type_1': 'category',
            'event_name_2': 'category',
            'event_type_2': 'category',
            'snap_CA': np.uint8,
            'snap_TX': np.uint8,
            'snap_WI': np.uint8,
        }
        cal = pd.read_csv(f'{path}/calendar.csv', 
                          dtype=cal_dtypes, 
                          usecols=list(cal_dtypes.keys()) + ['date'], 
                          parse_dates=['date'])
        
        event_cols = [k for k in cal_dtypes if k.startswith('event')]
        for col in event_cols:
            cal[col] = cal[col].cat.add_categories('nan').fillna('nan')
        
        # Prices
        prices_dtypes = {
            'store_id': 'category',
            'item_id': 'category',
            'wm_yr_wk': np.uint16,
            'sell_price': np.float32
        }

        prices = pd.read_csv(f'{path}/sell_prices.csv', 
                             dtype=prices_dtypes)
        
        # Sales
        sales_dtypes = {
            'id': 'category',
            'item_id': prices.item_id.dtype,
            'dept_id': 'category',
            'cat_id': 'category',
            'store_id': 'category',
            'state_id': 'category',
            **{f'd_{i+1}': np.float32 for i in range(1913)}
        }
        sales = pd.read_csv(f'{path}/sales_train_validation.csv', 
                            dtype=sales_dtypes)
        long = sales.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                          var_name='d', value_name='y')
        
        long['d'] = long['d'].astype(cal.d.dtype)
        long = long.merge(cal, on=['d'])
        long = long.merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'])
        long = long.drop(columns=['d', 'wm_yr_wk'])
        
        def first_nz_mask(values, index):
            """Return a boolean mask where the True starts at the first non-zero value."""
            mask = np.full(values.size, True)
            for idx, value in enumerate(values):
                if value == 0:
                    mask[idx] = False
                else:
                    break
            return mask
        
        long = long.sort_values(['id', 'date'], ignore_index=True)
        keep_mask = long.groupby('id')['y'].transform(first_nz_mask, engine='numba')
        long = long[keep_mask.astype(bool)]
        long.rename(columns={'id': 'unique_id', 'date': 'ds'}, inplace=True)
        Y_df = long.filter(items=['unique_id', 'ds', 'y'])
        cats = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
        S_df = long.filter(items=['unique_id'] + cats)
        S_df = S_df.drop_duplicates()
        X_df = long.drop(columns=['y'] + cats)
        
        if cache:
            pd.to_pickle((Y_df, X_df, S_df), file_cache)
        
        return Y_df, X_df, S_df

In [ ]:
Y_df, X_df, S_df = M5.load('./data')

## Test number of series

In [ ]:
n_series = 30_490
assert Y_df['unique_id'].unique().size == n_series
assert X_df['unique_id'].unique().size == n_series
assert S_df.shape[0] == 30_490

In [ ]:
Y_df.head()

,unique_id,ds,y
5,HOBBIES_1_001_CA_1_validation,2013-07-18,1.0
6,HOBBIES_1_001_CA_1_validation,2013-07-19,0.0
7,HOBBIES_1_001_CA_1_validation,2013-07-20,0.0
8,HOBBIES_1_001_CA_1_validation,2013-07-21,0.0
9,HOBBIES_1_001_CA_1_validation,2013-07-22,0.0


In [ ]:
X_df.head()

,unique_id,ds,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
5,HOBBIES_1_001_CA_1_validation,2013-07-18,nan,nan,nan,nan,0,0,0,9.58
6,HOBBIES_1_001_CA_1_validation,2013-07-19,nan,nan,nan,nan,0,0,0,9.58
7,HOBBIES_1_001_CA_1_validation,2013-07-20,nan,nan,nan,nan,0,0,0,9.58
8,HOBBIES_1_001_CA_1_validation,2013-07-21,nan,nan,nan,nan,0,0,0,9.58
9,HOBBIES_1_001_CA_1_validation,2013-07-22,nan,nan,nan,nan,0,0,0,9.58


In [ ]:
S_df.head()

,unique_id,item_id,dept_id,cat_id,store_id,state_id
5,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA
1020,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA
2796,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA
3605,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA
5482,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA


# Kaggle-Competition-M5 References

The evaluation metric of the Favorita Kaggle competition was the normalized weighted root mean squared logarithmic error (NWRMSLE).
Perishable items have a score weight of 1.25; otherwise, the weight is 1.0.

$$ NWRMSLE = \sqrt{\frac{\sum^{n}_{i=1} w_{i}\left(log(\hat{y}_{i}+1)  - log(y_{i}+1)\right)^{2}}{\sum^{n}_{i=1} w_{i}}}$$

Kaggle Competition Forecasting Methods                                                              | 16D ahead NWRMSLE
:-------------------------------------------------------------------------------------------------: | :-------: 
[LGBM](https://www.kaggle.com/shixw125/1st-place-lgb-model-public-0-506-private-0-511/comments) [1] | 0.5091   | 
[Seq2Seq WaveNet](https://arxiv.org/abs/1803.04037) [2]                                             | 0.5129   |


1.	[Corporación Favorita. Corporación favorita grocery sales forecasting. Kaggle Competition Leaderboard, 2018.](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/leaderboard)
2.	[Glib Kechyn, Lucius Yu, Yangguang Zang, and Svyatoslav Kechyn.  Sales forecasting using wavenet within the framework of the Favorita Kaggle competition. Computing Research Repository, abs/1803.04037, 2018](https://arxiv.org/abs/1803.04037).